In [1]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.utils.data import DataLoader, random_split

import torch.optim.lr_scheduler as scheduler
import torch.optim as optim
from datetime import datetime
import random


import torch.nn as nn
import torchvision.models as models

In [2]:
image_transforms = transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.Resize((128, 128)),               # Resize images to 128x128 pixels
    transforms.RandomHorizontalFlip(p=0.5),       # Randomly flip images horizontally with a probability of 0.5
    transforms.RandomVerticalFlip(p=0.5),         # Randomly flip images vertically with a probability of 0.5
    transforms.ToTensor()                         # Convert PIL Image to PyTorch Tensor
])

In [3]:
class AnimalDataset(Dataset):

    def __init__(self, rootpath, transform=None):

        self.rootpath = rootpath
        self.transform = transform
        self.allLabels = {}

        self.name1 = os.listdir(rootpath + 'cane/') 
        self.name2 = os.listdir(rootpath + 'cavallo/') 
        self.name3 = os.listdir(rootpath + 'elefante/') 
        self.name4 = os.listdir(rootpath + 'farfalla/') 
        self.name5 = os.listdir(rootpath + 'gallina/')
        self.name6 = os.listdir(rootpath + 'gatto/')
        self.name7 = os.listdir(rootpath + 'mucca/')
        self.name8 = os.listdir(rootpath + 'pecora/')
        self.name9 = os.listdir(rootpath + 'ragno/') 
        self.name10  = os.listdir(rootpath + 'scoiattolo/')


        for name in self.name1:
            self.allLabels[name] = 0
        for name in self.name2:
            self.allLabels[name] = 1
        for name in self.name3:
            self.allLabels[name] = 2
        for name in self.name4:
            self.allLabels[name] = 3
        for name in self.name5:
            self.allLabels[name] = 4
        for name in self.name6:
            self.allLabels[name] = 5
        for name in self.name7:
            self.allLabels[name] = 6
        for name in self.name8:
            self.allLabels[name] = 7
        for name in self.name9:
            self.allLabels[name] = 8
        for name in self.name10:
            self.allLabels[name] = 9

        self.allNames = list(self.allLabels.keys())
        random.shuffle(self.allNames)
        
        self.folders = ('cane/','cavallo/','elefante/', 'farfalla/', 'gallina/', 'gatto/','mucca/','pecora/', 'ragno/','scoiattolo/')

        self.count = 0

        self.allNames2 = []

    def __len__(self):
        return len(self.allNames)

    def __getitem__(self, index):        
        
        imageName = self.allNames[index]
        label = self.allLabels[imageName]        
        currentFolder = self.folders[label]
        img = Image.open(self.rootpath + currentFolder + imageName)

        if(self.transform):
            img = self.transform(img)

            
        return (img, label)

        

In [13]:
rootpath = 'data/archive/raw-img/'

import os 
os.listdir('data/archive/raw-img/')
os.listdir(rootpath + 'cane/')

['OIP-7Kyjub53-QqanyWanuJpOwHaHx.jpeg',
 'OIP-L3TRIOoPlmb3isOmx9DYDwHaEK.jpeg',
 'OIP-bTKnRAJfpOhQhD4AC4ZjZAAAAA.jpeg',
 'OIP-nj_6eSbDiI1jXD7cPD4i7wHaFr.jpeg',
 'OIP-sH7u4NwO_gj427Z0cw3C5gHaEX.jpeg',
 'OIP-KpXLr2QOZBB2ob4jLqQ_5AHaE8.jpeg',
 'OIP-5hI4KOsaeyvSjy7_LiTJCgHaFm.jpeg',
 'OIP-qgoh-WlYIKPSHlnkIp94BwHaHu.jpeg',
 'OIP-P3zTu-bZLnRAvvups-MEFQHaE6.jpeg',
 'OIP-EHAxcD8ywgCMF5xhF6LvUwHaEo.jpeg',
 'OIP-11I4h3JoOPyySDRiXsejsAHaFj.jpeg',
 'OIP-bqi9yp1JEHREY-Vn9ac23AHaNK.jpeg',
 'OIP-Ab97v-nnhG6AXLw1cAMgkQHaHa.jpeg',
 'OIP-ugSBKd8y_G2Wan07pBcoiwHaFj.jpeg',
 'OIP-gzs_Wk8UHWWmo1y1MEzhMgHaHu.jpeg',
 'OIP-lwIyWubwpiLRqzdbPUhdCgHaE7.jpeg',
 'OIP-H8TLN_kT4s19RFXqoJy92AHaEo.jpeg',
 'OIP-yz6ocLK9Ok8jCFEu3ebnJQHaFk.jpeg',
 'OIP-xxM7c6-_YFR317AF0A8NmgHaHN.jpeg',
 'OIP-yVrt8b9H133dZtyxCyjOFwHaFj.jpeg',
 'OIP-b73sEEY3g9wzdr2VlzZGtwHaFG.jpeg',
 'OIP-tfEfWdrWNEz1VNnXsYUMbAHaLH.jpeg',
 'OIP-BRsUa__xoqGpTyH7ySPZ5QHaJ-.jpeg',
 'OIP-76-YcKQApRKh-XA9eakmtQAAAA.jpeg',
 'OIP-nR9Hw3Ypa5IIu6k7Q54BnQHaE7.jpeg',


In [14]:
dataset = AnimalDataset(rootpath, transform = image_transforms)
print(len(dataset))

25840


In [15]:
trainset, valset, testset = random_split(dataset, [15000, 5000, 5840])

batch_size = 64

trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
valloader = DataLoader(valset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=True)

In [16]:
num_train_batches = len(trainloader)
num_validation_batches = len(valloader)
num_test_batches = len(testloader)

In [17]:
dataset.count

0

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device used for training is ",device)

Device used for training is  cpu


In [ ]:
model = models.resnet18(weights='IMAGENET1K_V1')
model.fc = nn.Linear(512, 10)

In [ ]:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
for param in model.fc.parameters():
    param.requires_grad = True

In [ ]:
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
frozen    = sum(p.numel() for p in model.parameters() if not p.requires_grad)
print(f"Trainable params : {trainable:,}")
print(f"Frozen params    : {frozen:,}")


In [20]:
!pip install torch-summary

In [21]:
from torchsummary import summary
summary(model, (3, 128, 128))

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 64, 64, 64]          9,408
├─BatchNorm2d: 1-2                       [-1, 64, 64, 64]          128
├─ReLU: 1-3                              [-1, 64, 64, 64]          --
├─MaxPool2d: 1-4                         [-1, 64, 32, 32]          --
├─Sequential: 1-5                        [-1, 64, 32, 32]          --
|    └─BasicBlock: 2-1                   [-1, 64, 32, 32]          --
|    |    └─Conv2d: 3-1                  [-1, 64, 32, 32]          36,864
|    |    └─BatchNorm2d: 3-2             [-1, 64, 32, 32]          128
|    |    └─ReLU: 3-3                    [-1, 64, 32, 32]          --
|    |    └─Conv2d: 3-4                  [-1, 64, 32, 32]          36,864
|    |    └─BatchNorm2d: 3-5             [-1, 64, 32, 32]          128
|    |    └─ReLU: 3-6                    [-1, 64, 32, 32]          --
|    └─BasicBlock: 2-2                   [-1, 64, 32, 32]          --
|

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 64, 64, 64]          9,408
├─BatchNorm2d: 1-2                       [-1, 64, 64, 64]          128
├─ReLU: 1-3                              [-1, 64, 64, 64]          --
├─MaxPool2d: 1-4                         [-1, 64, 32, 32]          --
├─Sequential: 1-5                        [-1, 64, 32, 32]          --
|    └─BasicBlock: 2-1                   [-1, 64, 32, 32]          --
|    |    └─Conv2d: 3-1                  [-1, 64, 32, 32]          36,864
|    |    └─BatchNorm2d: 3-2             [-1, 64, 32, 32]          128
|    |    └─ReLU: 3-3                    [-1, 64, 32, 32]          --
|    |    └─Conv2d: 3-4                  [-1, 64, 32, 32]          36,864
|    |    └─BatchNorm2d: 3-5             [-1, 64, 32, 32]          128
|    |    └─ReLU: 3-6                    [-1, 64, 32, 32]          --
|    └─BasicBlock: 2-2                   [-1, 64, 32, 32]          --
|

In [ ]:
# input1 = torch.rand(64, 3, 128,128)
# output = model(input1)
# print('final shape : ',output.shape)

In [22]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [23]:
lossFunction = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
decayLR = scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
sig = nn.Sigmoid()

In [24]:
epochs = 10

losses=[]
vacc = []
vlosses = []

for j in range(epochs):
    
    epoch_start = datetime.now()
    
    add_loss = 0.0
    run_loss2 = 0
    
    for i,data in enumerate(trainloader):
        
        image, label = data            
        image = image.to(device)
        label = label.to(device)
    
        optimizer.zero_grad()
    
        output = model(image)

        # print("output shape : ", output.shape)
    
        loss = lossFunction(output, label)
        
        add_loss += loss.item()
        run_loss2 += loss.item()
        
        loss.backward()        
        optimizer.step()
        
        if(i % 100 == 99):
            print('Epoch %d, Batch %d, Loss %.8f ' % (j+1, i+1, add_loss / 100))
            add_loss = 0.0    
    
    losses.append(run_loss2 / num_train_batches)
    
    print('------------')
    print('Validating')
    print('------------')
    
    total = 0
    correct = 0
    vrun_loss=0
    
    model.eval()
    
    with torch.no_grad():
        
        add_vloss = 0.0
        
        for k, vdata in enumerate(valloader):
            
            val_image, val_label = vdata
            
            val_image = val_image.to(device)
            val_label = val_label.to(device)
            
            val_output = model(val_image)
            
            vloss = lossFunction(val_output, val_label)
            
            add_vloss += vloss.item()
            vrun_loss += vloss.item()
            
            if(k%100 == 99):
                print('Validation loss : ', add_vloss / 100)
                add_vloss = 0.0
            
            class_probability, class_prediction = torch.max(val_output, 1)
            
            total += len(val_label)
            
            correct += (class_prediction == val_label).sum().item()
            
        val_accuracy = correct / total
        
        vlosses.append(vrun_loss / num_validation_batches)
        vacc.append(val_accuracy)
        print('---------')
        print('Correct : ', correct)
        print('Total : ', total)
        print('Final Validation accuracy : ', val_accuracy)
        print('---------')
        epoch_end = datetime.now()
        print('Epoch time : ', (epoch_end - epoch_start).total_seconds())
        print('---------------------------------')
        
    model.train()
    decayLR.step()
    
    print('Previous Learning Rate : ', decayLR.get_last_lr())
    print('------------------------')

KeyboardInterrupt: 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.style.use('seaborn-bright')

fig = plt.figure()

ax = plt.axes()

x_values = range(epochs)

losses2 = torch.tensor(losses).cpu()
vlosses2 = torch.tensor(vlosses).cpu()
vacc2 = torch.tensor(vacc).cpu()

ax.plot(x_values, losses2, color='blue',  linewidth=2, label='Training Loss' )
ax.plot(x_values, vlosses2, color='red',  linewidth=2, label='Validation Loss')
ax.plot(x_values, vacc2, color='green',  linewidth=2, label='Validation Accuracy')


plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Loss/Accuracy")
plt.tight_layout()

In [ ]:
# softmax = nn.Softmax(dim=1)

correct = 0
total = 0


with torch.no_grad():
    
    for i, data in enumerate(testloader):
        
        test_image, test_label = data
       
        test_image = test_image.to(device)
        test_label = test_label.to(device)      
        
        test_output = model(test_image)
        
        class_probability, class_prediction = torch.max(test_output, 1)

        total += len(test_label)
        
        correct += (class_prediction == test_label).sum().item()
        
    final_test_accuracy = correct/total
    
    print('Correct : ', correct)
    print('Total : ', total)
    print('Test accuracy is ', final_test_accuracy)

In [ ]:
!pip install torchcam

In [ ]:
from torchvision.io.image import read_image
from torchvision.transforms.functional import normalize, resize, to_pil_image
from torchcam.methods import GradCAM

In [ ]:
img = read_image("data/archive/raw-img/elefante/OIP---LeldVL441fx5S66TGgVQAAAA.jpeg")
layerToFocus = "layer4"

cam = GradCAM(model, layerToFocus)

activations, class_idx = cam(img.unsqueeze(0).to(device))

print(activations.shape)
print(class_idx)

In [25]:
# Preprocess it for your chosen model
input_tensor = normalize(resize(img, (128, 128)) / 255., [0.5, 0.5, 0.5], [0.2, 0.2, 0.2])

input_tensor = input_tensor.to(device)
with GradCAM(model, target_layer = layerToFocus) as cam_extractor:
  # Preprocess your data and feed it to the model
  out = model(input_tensor.unsqueeze(0))
  # Retrieve the CAM by passing the class index and the model output
  activation_map = cam_extractor(out.squeeze(0).argmax().item(), out)

NameError: name 'normalize' is not defined

In [ ]:
import matplotlib.pyplot as plt
from torchcam.utils import overlay_mask

# Resize the CAM and overlay it
result = overlay_mask(to_pil_image(img), to_pil_image(activation_map[0].squeeze(0), mode='F'), alpha=0.5)
# Display it
plt.imshow(result); plt.axis('off'); plt.tight_layout(); plt.show()